#### AI-Enhanced Q&A Content Generator and Chatbot
Business Problem:
 - Manual generation of course Q&A content.

• Objective:
 - Automate Q&A creation and minimize plagiarism.

• Approach:
- Developed a real-time Q&A generator system using Gemini-Pro, LangChain, and Google Generative
AI Embedding.

• Technologies Used:
 - LangChain,Generative AI,Google Gemini,LLM,Transformers


In [27]:
!pip install -q langchain_community
!pip install -q langchain_google_genai
!pip install -q load_dotenv
!pip install -q PyPDF2
!pip install -q  faiss-gpu

### **Step 2: Import Libraries**

In [28]:
import os

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")

In [29]:
import os
#import streamlit as st

#PdfReader

from PyPDF2 import PdfReader #PdfReader class from PyPDF2 to read PDF files
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # RecursiveCharacterTextSplitter for breaking down long texts into smaller chunks
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GoogleGenerativeAIEmbeddings to create embeddings using Google’s generative AI
import google.generativeai as genai # google.generativeai module for accessing Google’s generative AI capabilities
from langchain.vectorstores import FAISS#FAISS (Facebook AI Similarity Search) for efficient similarity search of dense vectors
#ChatGoogleGenerativeAI for building chat applications using Google’s generative AI models
from langchain_google_genai import ChatGoogleGenerativeAI  # For chat functionality
from langchain.chains.question_answering import load_qa_chain#load_qa_chain to create a question-answering pipeline
from langchain.prompts import PromptTemplate#PromptTemplate for constructing structured prompts for language models
from dotenv import load_dotenv, find_dotenv#oad_dotenv and find_dotenv from dotenv for managing environment variables


### **Step 3: Load Environment Variables**

In [30]:
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
print("Google API Key Loaded.")  # Check if the key is loaded successfully


Google API Key Loaded.


### **Step 4: Define PDF Text Extraction**

In [31]:
# Function to read text from a PDF file
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

# Example usage
pdf_path = "/content/attention.pdf"  #PDF file path
extracted_text = get_pdf_text([pdf_path])  # Extract text from the PDF



Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experime

In [ ]:
print(extracted_text)  # Print extracted text to see the output

### **Step 5: Split the Extracted Text into Chunks**

In [32]:
# Split text into the chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
chunks = text_splitter.split_text(extracted_text)

# Print the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)



Chunk 1:
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely.

In [33]:
##Length of the chunks
print(len(chunks))

5


### **Step 6: Create and Save the FAISS Index**

In [34]:
# Create embeddings and store them in a FAISS index
embeddings = GoogleGenerativeAIEmbeddings(api_key = google_api_key,model="models/embedding-001")
vector_store = FAISS.from_texts(chunks, embedding = embeddings)
vector_store.save_local("faiss_index")  # Save the index locally
print("FAISS index created and saved.")


FAISS index created and saved.


### **Step 7: Prepare the Conversational Chain**

In [35]:
#Creating the prompt template for the question-answering chain
prompt_template = """Answer the Question as Detailed as possible from the provided context,
                     make sure to provide all the details, if the answer not in the provided context just say "answer is not available in the context",
                     don't provide the wrong answer\n\n
                     Context: \n{context}\n
                     Question: \n{question}\n
                     Answer:"""

# Initialize the Chat model
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
prompt = PromptTemplate(template = prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(model, chain_type="stuff", prompt = prompt)
print("Conversational chain initialized.")



Conversational chain initialized.


In [36]:
prompt_template_mcq = """
You are an expert question paper creator specializing in technical subjects.
Generate {batch_size} unique multiple-choice questions (MCQs) based on the provided context extracted from the PDF. Ensure that the questions are completely original and free from plagiarism.
Each question should be accompanied by four answer choices labeled (A), (B), (C), and (D).

The format for the MCQs should be as follows:
<Question> (A) <Option A> (B) <Option B> (C) <Option C> (D) <Option D>

Context: {context}
"""

# Initialize the Chat model
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# Create the PromptTemplate instance
prompt = PromptTemplate(template=prompt_template_mcq, input_variables=["batch_size", "context"])

# Load the QA chain with the PromptTemplate instance
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
print("Conversational chain initialized.")


Conversational chain initialized.


### **Step 8: User Input and Q&A Processing**

In [37]:
# Ask the user for the number of questions and the topic
batch_size = int(input("How many questions would you like to generate? "))  # e.g., 10
user_question = input("Ask a Question from the PDF Files: ")  # e.g., "Create  Questions on Transformers"

# Load the FAISS index
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Perform similarity search to find relevant documents
docs = new_db.similarity_search(user_question)

# Combine the documents' content for context
context = " ".join([doc.page_content for doc in docs])

# Generate response based on user question, context, and batch_size
response = chain(
    {"input_documents": docs, "batch_size": batch_size, "context": context},
    return_only_outputs=True
)

# Print the generated response
print("Reply:", response.get("output_text", "No output generated."))


How many questions would you like to generate? 10
Ask a Question from the PDF Files: transformers
Reply: **1. What is the primary purpose of the Transformer model?**
(A) To improve the efficiency of recurrent neural networks
(B) To replace recurrent and convolutional layers with attention mechanisms
(C) To enhance the performance of machine translation tasks
(D) To introduce a novel approach to sequence transduction

**2. Which of the following is NOT a key component of the Transformer architecture?**
(A) Encoder and decoder stacks
(B) Self-attention layers
(C) Convolutional layers
(D) Positional encodings

**3. What is the main advantage of using self-attention layers over recurrent layers?**
(A) Faster computational speed
(B) Improved parallelization
(C) Reduced path length for long-range dependencies
(D) All of the above

**4. How does the Transformer model handle the problem of sequential computation?**
(A) By using a stack of convolutional layers
(B) By employing recurrent neural 

### **Chat With PDF**

In [37]:
# Prompt Template for to Chat with PDF
pdf_chat_prompt_template = """
You are an AI assistant that can only answer questions based on the content of a provided PDF document.
Do not provide any answers that are not in the document. If the answer is not in the PDF, simply respond with:
"The answer is not available in the provided PDF."

Use the following context extracted from the PDF to answer the user's question:

Context:
{context}

User's Question:
{question}

Answer:
"""

In [ ]:
# Initialize the Chat model
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)


In [ ]:
# Create the PromptTemplate
prompt = PromptTemplate(template=pdf_chat_prompt_template, input_variables=["context", "question"])


In [ ]:
# Load the QA chain with the PromptTemplate instance
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)


In [ ]:
# Example user interaction
user_question = input("Ask a question based on the PDF content: ")

# Combine the documents content for context
context = " ".join([doc.page_content for doc in docs])  # docs' is the  FAISS similarity search result,from previous


In [41]:
# Generate response based on user question and context
response = chain(
    {"input_documents": docs, "question": user_question, "context": context},
    return_only_outputs=True
)

Ask a question based on the PDF content: what is thsi pdf conatins
Reply: The provided PDF contains information about the Transformer, a neural network architecture for sequence transduction tasks such as machine translation. The Transformer is based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train.


In [38]:

# Print the generated response
print("Reply:", response.get("output_text", "No output generated."))